In [15]:
import requests
url = f'https://apipubaws.tcbs.com.vn/tcanalysis/v1/finance/TCB/cashflowasdasdsa'
requests.get(url, params={'yearly': 1, 'isAll': True}).json()

{'timestamp': 1686221688629,
 'status': 404,
 'error': 'Not Found',
 'path': '/p/v1/finance/TCB/cashflowasdasdsa'}

In [16]:
from multiprocessing import Process
import sys

rocket = 0

def func1():
    global rocket
    print('start func1')
    while rocket < 10000000:
        rocket += 1
    print('end func1')

def func2():
    global rocket
    print('start func2')
    while rocket < 10000000:
        rocket += 1
    print('end func2')

if __name__=='__main__':
    p1 = Process(target = func1)
    p1.start()
    p2 = Process(target = func2)
    p2.start()

start func1
start func2


In [2]:
pip install vnstock openpyxl excel2json

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests, pandas as pd
from io import BytesIO
from vnstock import *

In [ ]:
os.chdir("models")
for f in glob.glob("*.sql"):

In [1]:
import sqlparse, glob, os
def query_uppercase():
    for f in glob.glob("./*.sql"):
        f.open()
        query = sqlparse.format(f.read(), reindent=False, keyword_case='upper')
        f.write(query)
        f.close()
    

In [2]:
query_uppercase()

In [56]:
import re, sqlparse


def create_staging_script(df, name):
    query = """
        select
    """

    for columns in df.columns:
        if not any(char.isupper() for char in columns):
            query += f'{columns}, \n'
        else:
            new_column_name = re.sub( "(?<!^)(?=[A-Z]|[0-9])", "_", columns).lower()
            query += f'{columns} as {new_column_name},\n'

    query = query[:-3]
    query += f'\n'
    # query += 'from{{source(\'' + name + '\', \'' + name + '\') }}'
    query += f'from {{{{source(\'{name}\', \'{name}\')}}}}'

    query = sqlparse.format(query, reindent=True, keyword_case='upper')

    f = open(f'models/staging/native/stg_{name}.sql', "w")
    f.write(query)
    f.close()

    return None


In [57]:
create_staging_script(general_rating('TCB'), "general_rating")

In [65]:
def create_basic_dim_script(name):
    query = f'''with {name}  as (
    select * from {{{{ref ('stg_{name}')}}}}
),

final as (
    select *
    from {name} 
)

select * from final'''

    query = sqlparse.format(query, reindent=False, keyword_case='upper')

    f = open(f'models/marts/dim_{name}.sql', "w")
    f.write(query)
    f.close()

    return None

In [70]:
create_basic_dim_script('price_history')

In [20]:
def create_basic_dim_script(name):
    query = (
        f'with {name} as (\n'
        f'    select * from {{{{ref ('stg_{name}')}}}}\n'
        f'),\n'
        f'\n'
        f'final as (\n'
        f'    select *\n'
        f'    from {name}\n'
        f')\n'
        f'\n'
        f'select * from final\n'
    )

    f = open(f'models/marts/dim_{name}.sql', "w")
    f.write(query)
    f.close()

    return None

SyntaxError: invalid syntax. Perhaps you forgot a comma? (4204703233.py, line 3)

In [21]:
from pandas.io.json._table_schema import build_table_schema
import re
import json

# balancesheet = financial_flow('TCB', 'balancesheet','yearly')
# incomestatement = financial_flow('TCB', 'incomestatement','yearly')
# cashflow = financial_flow('TCB', 'cashflow','yearly')


def write_json_schema(df, file):
    schema = build_table_schema(df, index=False)
    schema = schema['fields']
    schema = str(schema)
    schema = re.sub("\[", "", schema)
    schema = re.sub("\]", "", schema)
    schema = re.sub("'", "\"", schema)
    schema = re.sub("{\"name\": ", "", schema)
    schema = re.sub(", \"type\"", ": {\"type\"", schema)


    header = """
    {
        "type": "object",
        "description": "",
        "properties": {
    """

    ending = """
        }
    }
    """

    return schema


In [23]:
generalrating = general_rating("TCB")
write_json_schema(generalrating, "source_tcbs_general_rating/schemas/general_rating.json")

'\n    {\n        "type": "object",\n        "description": "",\n        "properties": {\n    "stockRating": {"type": "number"}, "valuation": {"type": "number"}, "financialHealth": {"type": "number"}, "businessModel": {"type": "number"}, "businessOperation": {"type": "number"}, "rsRating": {"type": "number"}, "taScore": {"type": "number"}, "ticker": {"type": "string"}, "highestPrice": {"type": "number"}, "lowestPrice": {"type": "number"}, "priceChange3m": {"type": "number"}, "priceChange1y": {"type": "number"}, "beta": {"type": "number"}, "alpha": {"type": "number"}\n        }\n    }\n    '

In [24]:
from pandas.io.json._table_schema import build_table_schema
import re
import json

balancesheet = financial_flow('TCB', 'balancesheet','yearly')
incomestatement = financial_flow('TCB', 'incomestatement','yearly')
cashflow = financial_flow('TCB', 'cashflow','yearly')


def write_configured_catalog(df_list, file):
    final_json = """
    {
    "streams": [
    """

    for df in df_list:
        schema = build_table_schema(df)
        schema = schema['fields']
        schema = str(schema[1:])
        schema = re.sub("\[", "", schema)
        schema = re.sub("\]", "", schema)
        schema = re.sub("'", "\"", schema)
        schema = re.sub("{\"name\": ", "", schema)
        schema = re.sub(", \"type\"", ": {\"type\"", schema)


        header = """
        {
            "stream": {
            "name": "balance_sheet",
            "description": "Stock data price history",
            "json_schema": {
                "$schema": "http://json-schema.org/draft-04/schema#",
        """

        ending = """
            },
            "supported_sync_modes": ["full_refresh"]
            },
            "sync_mode": "full_refresh",
            "destination_sync_mode": "append"
        }
        """

        schema = header + schema + ending

        final_json += schema
    
    final_json += "] }"
    f = open(file, "w")
    f.write(final_json)
    f.close()

    return schema

# write_json_schema(balancesheet, "source_tcbs_financial_report/schemas/balance_sheet.json")
# write_json_schema(incomestatement, "source_tcbs_financial_report/schemas/incomestatement.json")
# write_json_schema(cashflow, "source_tcbs_financial_report/schemas/cashflow.json")

write_configured_catalog([generalrating], "integration_tests/configured_catalog.json")

'\n        {\n            "stream": {\n            "name": "balance_sheet",\n            "description": "Stock data price history",\n            "json_schema": {\n                "$schema": "http://json-schema.org/draft-04/schema#",\n        "stockRating": {"type": "number"}, "valuation": {"type": "number"}, "financialHealth": {"type": "number"}, "businessModel": {"type": "number"}, "businessOperation": {"type": "number"}, "rsRating": {"type": "number"}, "taScore": {"type": "number"}, "ticker": {"type": "string"}, "highestPrice": {"type": "number"}, "lowestPrice": {"type": "number"}, "priceChange3m": {"type": "number"}, "priceChange1y": {"type": "number"}, "beta": {"type": "number"}, "alpha": {"type": "number"}\n            },\n            "supported_sync_modes": ["full_refresh"]\n            },\n            "sync_mode": "full_refresh",\n            "destination_sync_mode": "append"\n        }\n        '

In [8]:
import requests
url = f'https://apipubaws.tcbs.com.vn/tcanalysis/v1/rating/TCB/general?fType=TICKER'
requests.get(url).json()

{'stockRating': 4.2,
 'valuation': 3.4,
 'financialHealth': 3.9,
 'businessModel': 3.4,
 'businessOperation': 4.7,
 'rsRating': 2.7,
 'taScore': 1.0,
 'stockRecommend': ['BID',
  'CTG',
  'SHB',
  'HDB',
  'MSB',
  'VCB',
  'TPB',
  'LPB',
  'MBB',
  'ACB'],
 'ticker': 'TCB',
 'highestPrice': 39550.0,
 'lowestPrice': 20700.0,
 'priceChange3m': 0.194,
 'priceChange1y': -0.096,
 'beta': 1.42,
 'alpha': 0.0007}